# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [ ]:
# Your task is to create a Python script using BeautifulSoup and pandas to scrape IMDb movie data based on user ratings and release dates.
# This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

from bs4 import BeautifulSoup
import pandas as pd
import requests
!pip install requests beautifulsoup4



In [ ]:

url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,"
response = requests.get(url,headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'})
response

In [ ]:
print(response.headers['Content-Type'])

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
type(soup)

In [ ]:
# print(soup.prettify())

In [ ]:
soup.find("title")

In [ ]:
soup.find_all('h3', class_="ipc-title__text")
print(soup.find_all('h3', class_="ipc-title__text"))

In [ ]:

name_element = soup.find_all("h3" , class_ = 'ipc-title__text')


In [ ]:
titles_lists = [i.get_text().split(".") for i in name_element]


In [ ]:
len(titles_lists)

In [ ]:
# Movie Nr: The number representing the movie’s position in the list.
number  = []
for i in range(50):
    number.append(titles_lists[i][0])

print (number)

In [ ]:
# Title: The title of the movie.


name  = []
for i in range(50):
    name.append(titles_lists[i][1])

print (name)


In [ ]:
# Year: The year the movie was released.
# Runtime (min): The duration of the movie in minutes.

year_runtime_element = soup.find_all( "span", class_="sc-b0691f29-8 ilsLEX dli-title-metadata-item")
year_runtime_element
year_runtime_lists = [i.get_text().split(".") for i in year_runtime_element]
year_runtime_lists
 

In [ ]:
year = []
runtime_interim = []
runtime =[]
for i in range (0, 50, 3):
    year.append(year_runtime_lists[i])
for i in range (1, 50, 3):
    runtime_interim.append(year_runtime_lists[i])

runtime_new = [time[0].replace('m', '') for time in runtime_interim]
print(runtime_new)

for i in runtime_new:
    if 'h' in i:
        if 'm' in i:
            hours, minutes = map(int, i.split('h '))
            total_minutes = hours * 60 + minutes
        else:
            total_minutes = int(i.split('h')[0]) * 60
    else:
        total_minutes = int(i.split('m')[0])
    runtime.append(total_minutes)

print(runtime)

In [ ]:
# <span aria-label="IMDb rating: 8.6" class="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating" data-testid="ratingGroup--imdb-rating"><svg width="24" height="24" xmlns="http://www.w3.org/2000/svg" class="ipc-icon ipc-icon--star-inline" viewBox="0 0 24 24" fill="currentColor" role="presentation"><path d="M12 20.1l5.82 3.682c1.066.675 2.37-.322 2.09-1.584l-1.543-6.926 5.146-4.667c.94-.85.435-2.465-.799-2.567l-6.773-.602L13.29.89a1.38 1.38 0 0 0-2.581 0l-2.65 6.53-6.774.602C.052 8.126-.453 9.74.486 10.59l5.147 4.666-1.542 6.926c-.28 1.262 1.023 2.26 2.09 1.585L12 20.099z"></path></svg>8.6<span class="ipc-rating-star--voteCount">&nbsp;(1.5M)</span></span>
rating_element = soup.find_all( "span", class_="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")
rating_element

In [ ]:
rating_element_lists = [i.get_text() for i in rating_element]
rating_element_lists


In [ ]:
import re
rating = []
votes = []

for i in rating_element_lists:
    rating_lev = re.search(r'\d+\.\d+', i).group()
    rating.append( rating_lev)
print(rating)

for i in rating_element_lists:
    match = re.search(r'\((.*?)\)', i)
    if match:
        value_in_parentheses = match.group(1)
        votes.append(value_in_parentheses)

print(votes)

In [ ]:
description_element = soup.find_all( "div" , class_="ipc-html-content-inner-div")
description_element
description_element_lists = ([i.get_text() for i in description_element])
description = description_element_lists
print(description)

In [3]:
def scrape_imdb(title_type, user_rating, start_date, end_date):
   
    url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,"
    response = requests.get(url,headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'})
    response
    type(soup)
    soup.find("title")
    soup.find_all('h3', class_="ipc-title__text")
    print(soup.find_all('h3', class_="ipc-title__text"))
    
    name_element = soup.find_all("h3" , class_ = 'ipc-title__text')
    
    titles_lists = [i.get_text().split(".") for i in name_element]
    len(titles_lists)
    
    number  = []
    for i in range(50):
        number.append(titles_lists[i][0])
    
    name  = []
    for i in range(50):
        name.append(titles_lists[i][1])
    
    year_runtime_element = soup.find_all( "span", class_="sc-b0691f29-8 ilsLEX dli-title-metadata-item")
    year_runtime_element
    year_runtime_lists = [i.get_text().split(".") for i in year_runtime_element]
    year_runtime_lists
    year = []
    runtime_interim = []
    runtime =[]
    for i in range (0, 50, 3):
        year.append(year_runtime_lists[i])
    for i in range (1, 50, 3):
        runtime_interim.append(year_runtime_lists[i])

    runtime_new = [time[0].replace('m', '') for time in runtime_interim]
    print(runtime_new)

    for i in runtime_new:
        if 'h' in i:
            if 'm' in i:
                hours, minutes = map(int, i.split('h '))
                total_minutes = hours * 60 + minutes
            else:
                total_minutes = int(i.split('h')[0]) * 60
        else:
            total_minutes = int(i.split('m')[0])
        runtime.append(total_minutes)
    
    rating_element = soup.find_all( "span", class_="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")
    rating_element
    rating_element_lists = [i.get_text() for i in rating_element]
    rating_element_lists
    import re
    rating = []
    votes = []

    for i in rating_element_lists:
        rating_lev = re.search(r'\d+\.\d+', i).group()
        rating.append( rating_lev)

    for i in rating_element_lists:
        match = re.search(r'\((.*?)\)', i)
        if match:
            value_in_parentheses = match.group(1)
            votes.append(value_in_parentheses)
    
    description_element = soup.find_all( "div" , class_="ipc-html-content-inner-div")
    description_element
    description_element_lists = ([i.get_text() for i in description_element])
    description = description_element_lists
    
    df = pd.DataFrame({
    'Title': name,
    'Year': year,
    'Rating': rating,
    'Runtime': runtime,
    'Description': description,
    'Votes': votes,
})

    return df


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here

In [ ]:
base_url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,"
start = 1
while start < 631:
    page_url = f"{base_url}&start={start}&ref_=adv_nxt"
    print(page_url)
    start += 50